# 🚀 Visual Odometry System Demo

This notebook demonstrates the Enhanced Visual Odometry System capabilities.

## Features Covered:
- Data loading and preprocessing
- Visual odometry algorithms
- Real-time processing
- Performance analysis
- Visualization

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Add backend to path
sys.path.append('../backend')

# Import our modules
from core.visual_odometry import (
    VisualOdometryPipeline, 
    CameraParams, 
    MonocularVO,
    FeatureDetector
)
from data.dataset_loader import DatasetManager
from utils.metrics import MetricsCollector
from utils.visualization import TrajectoryPlotter
from utils.logger import setup_logger

# Setup
logger = setup_logger("vo_demo")
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Initialize System Components

In [ ]:
# Initialize components
dataset_manager = DatasetManager(data_dir="../datasets")
metrics_collector = MetricsCollector()
plotter = TrajectoryPlotter()

# Camera parameters (typical values)
camera_params = CameraParams(
    fx=525.0,
    fy=525.0,
    cx=319.5,
    cy=239.5
)

print("✅ System components initialized")
print(f"📊 Camera intrinsics: fx={camera_params.fx}, fy={camera_params.fy}")
print(f"📍 Principal point: cx={camera_params.cx}, cy={camera_params.cy}")

## 2. Prepare Sample Data

In [ ]:
# Prepare sample data
print("📥 Preparing sample data...")
sample_path = dataset_manager.prepare_sample_data()
print(f"✅ Sample data created at: {sample_path}")

# Load the dataset
dataset = dataset_manager.load_dataset('sample')
print(f"📊 Dataset info:")
print(f"  - Type: {dataset['type']}")
print(f"  - Sequence: {dataset['sequence']}")
print(f"  - Number of frames: {dataset['num_frames']}")
print(f"  - Images path: {len(dataset['images'])} images")

## 3. Feature Detection Comparison

In [ ]:
# Compare different feature detectors
detectors = ['ORB', 'SIFT']
test_image_path = dataset['images'][0]
test_image = cv2.imread(test_image_path)

fig, axes = plt.subplots(1, len(detectors), figsize=(15, 5))
if len(detectors) == 1:
    axes = [axes]

for i, detector_type in enumerate(detectors):
    try:
        # Create detector
        detector = FeatureDetector(detector_type)
        
        # Detect features
        keypoints, descriptors = detector.detect_and_compute(test_image)
        
        # Draw keypoints
        img_with_keypoints = cv2.drawKeypoints(
            test_image, keypoints, None, color=(0, 255, 0), flags=0
        )
        
        # Convert BGR to RGB for matplotlib
        img_rgb = cv2.cvtColor(img_with_keypoints, cv2.COLOR_BGR2RGB)
        
        axes[i].imshow(img_rgb)
        axes[i].set_title(f'{detector_type}: {len(keypoints)} keypoints')
        axes[i].axis('off')
        
        print(f"✅ {detector_type}: {len(keypoints)} keypoints detected")
        
    except Exception as e:
        print(f"❌ {detector_type} failed: {e}")
        axes[i].text(0.5, 0.5, f'{detector_type}\nFailed', 
                    ha='center', va='center', transform=axes[i].transAxes)
        axes[i].axis('off')

plt.tight_layout()
plt.show()

## 4. Visual Odometry Processing

In [ ]:
# Create visual odometry pipeline
vo_pipeline = VisualOdometryPipeline(
    camera_params=camera_params,
    mode='mono',
    detector_type='ORB'
)

print("🚀 Starting visual odometry processing...")

# Start metrics collection
session_id = metrics_collector.start_session(
    dataset='sample',
    sequence='synthetic',
    algorithm='ORB_monocular'
)

# Process images
results = []
processing_times = []

for i, image_path in enumerate(dataset['images'][:20]):  # Process first 20 frames
    start_time = time.time()
    
    # Load and process image
    image = cv2.imread(image_path)
    if image is None:
        continue
    
    # Process frame
    pose, stats = vo_pipeline.vo.process_frame(image)
    
    processing_time = time.time() - start_time
    processing_times.append(processing_time)
    
    # Record metrics
    metrics_collector.record_frame(
        frame_number=i,
        processing_time=processing_time,
        keypoints=stats.get('keypoints_count', 0),
        matches=stats.get('matches_count', 0),
        inliers=stats.get('inliers_count', 0),
        position=pose.t.flatten().tolist(),
        rotation=pose.R.tolist()
    )
    
    results.append({
        'frame': i,
        'pose': pose,
        'stats': stats,
        'processing_time': processing_time
    })
    
    if i % 5 == 0:
        print(f"📊 Frame {i}: {stats.get('keypoints_count', 0)} keypoints, "
              f"{stats.get('matches_count', 0)} matches, "
              f"{processing_time:.3f}s")

# End metrics collection
session = metrics_collector.end_session(success=True)
print(f"✅ Processing completed: {len(results)} frames")
print(f"⚡ Average FPS: {1.0/np.mean(processing_times):.2f}")

## 5. Trajectory Visualization

In [ ]:
# Get trajectory data
trajectory = vo_pipeline.get_trajectory_array()

if len(trajectory) > 0:
    print(f"📈 Trajectory contains {len(trajectory)} poses")
    print(f"🎯 Start position: [{trajectory[0, 0]:.2f}, {trajectory[0, 1]:.2f}, {trajectory[0, 2]:.2f}]")
    print(f"🏁 End position: [{trajectory[-1, 0]:.2f}, {trajectory[-1, 1]:.2f}, {trajectory[-1, 2]:.2f}]")
    
    # Calculate trajectory length
    distances = np.linalg.norm(np.diff(trajectory, axis=0), axis=1)
    total_distance = np.sum(distances)
    print(f"📏 Total distance traveled: {total_distance:.2f} meters")
    
    # Plot 2D trajectory
    plt.figure(figsize=(12, 8))
    plt.plot(trajectory[:, 0], trajectory[:, 2], 'b-', linewidth=2, label='Estimated trajectory')
    plt.scatter(trajectory[0, 0], trajectory[0, 2], c='green', s=100, marker='o', label='Start')
    plt.scatter(trajectory[-1, 0], trajectory[-1, 2], c='red', s=100, marker='s', label='End')
    
    plt.xlabel('X (meters)')
    plt.ylabel('Z (meters)')
    plt.title('Visual Odometry Trajectory (Top View)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.show()
    
    # Plot 3D trajectory
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], 'b-', linewidth=2)
    ax.scatter(trajectory[0, 0], trajectory[0, 1], trajectory[0, 2], 
              c='green', s=100, marker='o', label='Start')
    ax.scatter(trajectory[-1, 0], trajectory[-1, 1], trajectory[-1, 2], 
              c='red', s=100, marker='s', label='End')
    
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.set_zlabel('Z (meters)')
    ax.set_title('3D Visual Odometry Trajectory')
    ax.legend()
    plt.show()
    
else:
    print("⚠️ No trajectory data available")

## 6. Performance Analysis

In [ ]:
# Get session summary
summary = metrics_collector.get_session_summary(session_id)

print("📊 Performance Summary:")
print(f"  Session ID: {summary['session_id']}")
print(f"  Algorithm: {summary['algorithm']}")
print(f"  Total frames: {summary['total_frames']}")
print(f"  Duration: {summary['duration_seconds']:.2f} seconds")
print(f"  Average FPS: {summary['fps']:.2f}")
print(f"  Trajectory length: {summary['trajectory_length']:.2f} meters")

print("\n⏱️ Processing Time Statistics:")
proc_stats = summary['processing_time']
print(f"  Mean: {proc_stats['mean']*1000:.2f} ms")
print(f"  Std: {proc_stats['std']*1000:.2f} ms")
print(f"  Min: {proc_stats['min']*1000:.2f} ms")
print(f"  Max: {proc_stats['max']*1000:.2f} ms")

print("\n🎯 Feature Statistics:")
kp_stats = summary['keypoints']
match_stats = summary['matches']
print(f"  Keypoints - Mean: {kp_stats['mean']:.1f}, Std: {kp_stats['std']:.1f}")
print(f"  Matches - Mean: {match_stats['mean']:.1f}, Std: {match_stats['std']:.1f}")
print(f"  Match ratio: {match_stats['mean']/kp_stats['mean']*100:.1f}%")

## 7. Metrics Visualization

In [ ]:
# Extract metrics for plotting
frame_numbers = [r['frame'] for r in results]
keypoints_counts = [r['stats'].get('keypoints_count', 0) for r in results]
matches_counts = [r['stats'].get('matches_count', 0) for r in results]
processing_times_ms = [r['processing_time'] * 1000 for r in results]

# Create metrics plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Visual Odometry Performance Metrics', fontsize=16)

# Keypoints over time
axes[0, 0].plot(frame_numbers, keypoints_counts, 'b-', linewidth=2)
axes[0, 0].set_title('Keypoints Detected per Frame')
axes[0, 0].set_xlabel('Frame Number')
axes[0, 0].set_ylabel('Keypoints Count')
axes[0, 0].grid(True, alpha=0.3)

# Matches over time
axes[0, 1].plot(frame_numbers, matches_counts, 'g-', linewidth=2)
axes[0, 1].set_title('Feature Matches per Frame')
axes[0, 1].set_xlabel('Frame Number')
axes[0, 1].set_ylabel('Matches Count')
axes[0, 1].grid(True, alpha=0.3)

# Processing time
axes[1, 0].plot(frame_numbers, processing_times_ms, 'r-', linewidth=2)
axes[1, 0].set_title('Processing Time per Frame')
axes[1, 0].set_xlabel('Frame Number')
axes[1, 0].set_ylabel('Time (ms)')
axes[1, 0].grid(True, alpha=0.3)

# Match efficiency
match_ratios = [m/k if k > 0 else 0 for k, m in zip(keypoints_counts, matches_counts)]
axes[1, 1].plot(frame_numbers, match_ratios, 'purple', linewidth=2)
axes[1, 1].set_title('Match Efficiency (Matches/Keypoints)')
axes[1, 1].set_xlabel('Frame Number')
axes[1, 1].set_ylabel('Ratio')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Algorithm Comparison

In [ ]:
# Compare different algorithms
algorithms = ['ORB']  # Add 'SIFT' if available
comparison_results = {}

for algo in algorithms:
    try:
        print(f"🔄 Testing {algo} algorithm...")
        
        # Create new pipeline
        test_pipeline = VisualOdometryPipeline(
            camera_params=camera_params,
            mode='mono',
            detector_type=algo
        )
        
        # Start new session
        test_session_id = metrics_collector.start_session(
            dataset='sample',
            sequence='synthetic',
            algorithm=f'{algo}_comparison'
        )
        
        # Process subset of images
        test_images = dataset['images'][:10]  # First 10 frames
        start_time = time.time()
        
        for i, image_path in enumerate(test_images):
            image = cv2.imread(image_path)
            if image is not None:
                frame_start = time.time()
                pose, stats = test_pipeline.vo.process_frame(image)
                frame_time = time.time() - frame_start
                
                metrics_collector.record_frame(
                    frame_number=i,
                    processing_time=frame_time,
                    keypoints=stats.get('keypoints_count', 0),
                    matches=stats.get('matches_count', 0),
                    inliers=stats.get('inliers_count', 0),
                    position=pose.t.flatten().tolist(),
                    rotation=pose.R.tolist()
                )
        
        total_time = time.time() - start_time
        test_session = metrics_collector.end_session(success=True)
        test_summary = metrics_collector.get_session_summary(test_session_id)
        
        comparison_results[algo] = {
            'fps': test_summary['fps'],
            'avg_keypoints': test_summary['keypoints']['mean'],
            'avg_matches': test_summary['matches']['mean'],
            'trajectory_length': test_summary['trajectory_length']
        }
        
        print(f"  ✅ {algo}: {test_summary['fps']:.2f} FPS, "
              f"{test_summary['keypoints']['mean']:.1f} keypoints avg")
        
    except Exception as e:
        print(f"  ❌ {algo} failed: {e}")

# Display comparison
if comparison_results:
    print("\n📊 Algorithm Comparison:")
    for algo, results in comparison_results.items():
        print(f"\n{algo}:")
        print(f"  FPS: {results['fps']:.2f}")
        print(f"  Avg Keypoints: {results['avg_keypoints']:.1f}")
        print(f"  Avg Matches: {results['avg_matches']:.1f}")
        print(f"  Trajectory Length: {results['trajectory_length']:.2f}m")

## 9. Export Results

In [ ]:
# Export session data
output_dir = Path('../output')
output_dir.mkdir(exist_ok=True)

# Export metrics
export_file = output_dir / f"metrics_{session_id}.json"
metrics_collector.export_session(session_id, str(export_file))
print(f"📁 Metrics exported to: {export_file}")

# Save trajectory
if len(trajectory) > 0:
    trajectory_file = output_dir / f"trajectory_{session_id}.csv"
    np.savetxt(trajectory_file, trajectory, delimiter=',', 
               header='x,y,z', comments='', fmt='%.6f')
    print(f"📍 Trajectory saved to: {trajectory_file}")

print("\n✅ Demo completed successfully!")
print("\n🎯 Next Steps:")
print("  - Try different feature detectors (SIFT, SURF)")
print("  - Test with real camera data")
print("  - Experiment with stereo visual odometry")
print("  - Compare with ground truth trajectories")
print("  - Run the web interface for real-time visualization")